## Predict for test site images using the saved model

In [ ]:
!unzip /content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/planet_testsite_patches_png.zip -d /content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/
!unzip /content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/planet_testsite_patches_tiff.zip -d /content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/

In [ ]:
import glob

# search all files inside a specific folder
# *.* means file name with any extension
dir_path =  "/content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/planet_testsite_patches_png/*"

# Use glob to find all the files matching the pattern
test_images_names = glob.glob(dir_path, recursive=True)
test_images_names

In [ ]:
len(test_images_names)

In [ ]:
val_images = test_images_names

In [ ]:
# load the saved model

id2label = {0: 'background', 1: 'water'}
label2id = {label: id for id, label in id2label.items()}
num_labels = len(id2label)

model = SegformerForSemanticSegmentation.from_pretrained(
   '/content/drive/MyDrive/MA/Models/segformer_finetuned_water_planet_RGB_fullimage_2607',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)


feature_extractor = SegformerImageProcessor.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
import os
import numpy as np
import tifffile
import cv2
import matplotlib.pyplot as plt
import torch.nn.functional as nnf
from PIL import Image

def save_predictions(val_images, model, feature_extractor, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for i in range(len(val_images)):
        image_path = val_images[i]
        #mask_path = val_masks[i]

        image = tifffile.imread(image_path)
        #mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        print(f'Validation image #{i + 1}')

        inputs = np.moveaxis(image, -1, 0)
        inputs = feature_extractor(images=image, return_tensors='pt')

        outputs = model(**inputs)
        logits = outputs.logits

        # Rescale logits to original image size
        upsampled_logits = nnf.interpolate(
            logits,
            size=image.shape[:-1], # (height, width)
            mode='bilinear',
            align_corners=False
        )

        # Apply argmax on the class dimension
        pred_mask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()

        # Create the output file path with '_pred' attached to the original mask name
        image_name = os.path.basename(image_path)
        image_name_without_ext, ext = os.path.splitext(image_name)
        pred_mask_name = f"{image_name_without_ext}_pred.png"
        pred_mask_path = os.path.join(output_folder, pred_mask_name)

        # Save the predicted mask as a PNG file
        pred_mask_image = Image.fromarray(pred_mask.astype(np.uint8))
        pred_mask_image.save(pred_mask_path)

        # Display the images and masks
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

        ax1.imshow(image)
        ax1.set_title('Image')
        ax1.axis('Off')

        ax2.imshow(pred_mask, cmap='Blues')
        ax2.set_title('Predicted mask')
        ax2.axis('Off')

        plt.show()


output_folder = "/content/drive/MyDrive/boundary_demarcation/Planet/RESULTS/TEST_SITE/TestSite_Prediction_masks"

# Assume `model` and `feature_extractor` are defined and initialized
save_predictions(val_images, model, feature_extractor, output_folder)

In [ ]:
import os

directory = "/content/drive/MyDrive/boundary_demarcation/Planet/RESULTS/TEST_SITE/TestSite_Prediction_masks"
lst = os.listdir(directory) # your directory path
number_files = len(lst)
print( number_files)

## apply majority filter on the predictions

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters.rank import modal
from skimage.morphology import rectangle
import os
import glob

def apply_modal_filter(image, footprint_size=(3, 3)):
    # Apply the modal filter
    result = modal(image, rectangle(*footprint_size))
    return result

def process_images(input_dir, output_dir, footprint_size=(3, 3)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get all image files in the input directory
    image_paths = glob.glob(os.path.join(input_dir, '*.png'))
    # print(image_paths)

    for image_path in image_paths:
        # Read the input image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply the modal filter
        result = apply_modal_filter(image, footprint_size)
        result = apply_modal_filter(result, footprint_size)
        result = apply_modal_filter(result, footprint_size)

        # Save the filtered result as a JPEG image
        base_name = os.path.basename(image_path)
        output_path = os.path.join(output_dir, f"{os.path.splitext(base_name)[0]}_majority.png")
        cv2.imwrite(output_path, result)

        # Plot the original and filtered images
        plt.figure(figsize=(14, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(image, cmap='gray')
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(result, cmap='gray')
        plt.title('Filtered Image')
        plt.show()

# Paths
input_dir = "/content/drive/MyDrive/boundary_demarcation/Planet/Results/predicted_masks_test"
output_dir = '/content/drive/MyDrive/boundary_demarcation/Planet/Results/predicted_masks_majority_test'

# Process all images in the input directory
process_images(input_dir, output_dir)

## convert predicted masks (majority applied) to shapefile

### fetch projection and transform from TIFF image and add to shapefile

In [ ]:
!unzip /content/drive/MyDrive/boundary_demarcation/Planet/DATA_full/planet_patches_tiff.zip -d /content/drive/MyDrive/boundary_demarcation/Planet/DATA_full/planet_patches_tiff

In [ ]:
import cv2
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os
import rasterio
from rasterio.transform import Affine
from shapely.ops import transform
from pyproj import Transformer

# Function to read and process each mask image
def process_mask(mask_path):
    # Read the binary mask image
    binary_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Ensure the binary mask is binary (0 or 255)
    #_, binary_mask = cv2.threshold(binary_mask, 0.5, 1, cv2.THRESH_BINARY)

    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert valid contours to polygons
    polygons = []
    for cnt in contours:
        if len(cnt) >= 4:  # Ensure the contour has at least 4 points
            polygon = Polygon(cnt.reshape(-1, 2))
            polygons.append(polygon)

    return polygons

# Function to transform the polygon coordinates based on the affine transform
def apply_affine_transform(polygon, affine_transform):
    transformed_coords = [affine_transform * (x, y) for x, y in polygon.exterior.coords]
    return Polygon(transformed_coords)

# Directory where the mask images are saved
mask_dir = "/content/drive/MyDrive/boundary_demarcation/Planet/RESULTS/TEST_SITE/TestSite_Prediction_masks"
vec_dir  = "/content/drive/MyDrive/boundary_demarcation/Planet/RESULTS/TEST_SITE/TestSite_Predicted_vectors"
tiff_dir = "/content/drive/MyDrive/boundary_demarcation/Planet/DATA_Test/planet_testsite_patches_tiff"  # Directory where the TIFF files are saved

# List of mask image filenames
mask_filenames = [f for f in os.listdir(mask_dir) if f.endswith('.png')]

# Process each mask image and save as an individual shapefile
for mask_filename in mask_filenames:
    mask_path = os.path.join(mask_dir, mask_filename)
    polygons = process_mask(mask_path)

    # Corresponding TIFF filename (removing '_pred' from the mask filename)
    tiff_filename = mask_filename.replace('_pred.png', '') + '.tif'
    tiff_path = os.path.join(tiff_dir, tiff_filename)

    # Check if the TIFF file exists
    if not os.path.exists(tiff_path):
        print(f"TIFF file not found: {tiff_path}")
        continue

    # Read the projection information and affine transform from the TIFF file
    with rasterio.open(tiff_path) as src:
        crs = src.crs
        affine_transform = src.transform

    # Transform the polygons to the correct coordinates
    transformed_polygons = [apply_affine_transform(polygon, affine_transform) for polygon in polygons]

    # Initialize the transformer
    transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)

    # Convert the transformed polygons to the desired CRS (WGS 84)
    wgs84_polygons = [transform(transformer.transform, polygon) for polygon in transformed_polygons]

    # Save polygons to a GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=wgs84_polygons, crs='EPSG:4326')

    # Define output shapefile path
    output_shapefile_path = os.path.join(vec_dir, f'{os.path.splitext(mask_filename)[0]}.shp')

    # Save the GeoDataFrame to a shapefile
    gdf.to_file(output_shapefile_path)

    # Print confirmation
    print(f'Saved {output_shapefile_path}')

# Model Evaluation on Val Set

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import seaborn as sns

def compute_test_metrics(model, feature_extractor, test_images, test_masks, id2label):
    all_preds = []
    all_labels = []

    for i in range(len(test_images)):
        image_path = test_images[i]
        mask_path = test_masks[i]

        image = tifffile.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

        # Preprocess the image
        inputs = feature_extractor(images=image, return_tensors='pt')

        # Get the model output
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Rescale logits to original image size
            upsampled_logits = nn.functional.interpolate(
                logits,
                size=image.shape[:-1], # (height, width)
                mode='bilinear',
                align_corners=False
            )

            # Apply argmax on the class dimension
            pred_mask = upsampled_logits.argmax(dim=1)[0].cpu().numpy()

        # Flatten the masks for metric calculations
        all_preds.extend(pred_mask.flatten())
        all_labels.extend(mask.flatten())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds, labels=list(id2label.keys()))

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, labels=list(id2label.keys()), average=None)

    # Compute IoU
    iou_scores = {}
    for i, label in id2label.items():
        intersection = np.logical_and(all_labels == i, all_preds == i).sum()
        union = np.logical_or(all_labels == i, all_preds == i).sum()
        iou_scores[label] = intersection / union if union != 0 else 0

    print("\nConfusion Matrix:")
    print(conf_matrix)

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, labels=list(id2label.keys()), target_names=list(id2label.values())))

    print("\nIoU Scores:")
    for label, iou in iou_scores.items():
        print(f"{label}: {iou:.4f}")

    # Visualization of the confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt="d", xticklabels=id2label.values(), yticklabels=id2label.values())
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()


# Assuming you have already loaded your validation or test data
compute_test_metrics(model, feature_extractor, val_images, val_masks, id2label)

In [ ]:
# percentage confusion matrix

# Convert to row-wise percentages
conf_matrix_percent = conf_matrix.astype('float') / conf_matrix.sum(axis=1, keepdims=True) * 100

# Plot using seaborn
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_percent, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=['Background', 'Water'],
            yticklabels=['Background', 'Water'])

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (%)")
plt.tight_layout()
plt.show()
